# 스타 사진 데이터 전처리
- 남자 아이돌 100명, 여자 아이돌 100명, 기타 스타 10명
- 스타당 500개 이미지 수집 후 단체 사진이거나 마스크 사진 검수 후 삭제함
- 640 X 640 리사이즈 및 패딩
- YOLO WORLD 모델로 얼굴 Bounding Box 좌표 출력


## 1. 이미지 리사이즈

In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
from PIL import Image
import tensorflow as tf
from matplotlib import pyplot as plt
import matplotlib.cm as cm
from sklearn.model_selection import train_test_split
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive') # 구글 드라이브 마운트

Mounted at /content/drive


In [ ]:
# 이미지 불러오기
folder_path = '/content/drive/MyDrive/Colab Notebooks/PJ4_최애티콘/star_image'
label_names = os.listdir(folder_path) # 폴더명
# label_names.remove('.DS_Store') # Mac용
print(label_names)

In [ ]:
dataset = {} # {'스타명': 경로/파일명.jpg}
for label in os.listdir(folder_path):
    if label != '.DS_Store': # Mac용
        sub_path = folder_path+'/'+label+'/'
        dataset[label] = []
        for filename in os.listdir(sub_path):
            if filename != '.DS_Store': # Mac용
                dataset[label].append(sub_path+filename)

In [ ]:
a = dataset['naver_에스파 카리나'][0]
print(a[64:])
print(a)

naver_에스파 카리나/90.jpg
/content/drive/MyDrive/Colab Notebooks/PJ4_최애티콘/star_image/naver_에스파 카리나/90.jpg


### 이미지 리사이즈&패딩

In [ ]:
def resize_img(img_path, img_size=640): # resize 함수
    global img_re

    img = cv2.imread(img_path)

    if img is not None:
        if (img.shape[1] > img.shape[0]):
            ratio = img_size/img.shape[1]
        else:
            ratio = img_size/img.shape[0]

        img = cv2.resize(img, dsize=(0, 0), fx=ratio, fy=ratio, interpolation=cv2.INTER_LINEAR)

        # 그림 주변에 검은색으로 칠하기
        w, h = img.shape[1], img.shape[0]

        dw = (img_size-w)/2 # img_size와 w의 차이
        dh = (img_size-h)/2 # img_size와 h의 차이

        M = np.float32([[1,0,dw], [0,1,dh]])  #(2*3 이차원 행렬)
        img_re = cv2.warpAffine(img, M, (640, 640)) #이동변환
    return img_re

### 이미지 저장

In [ ]:
if not os.path.exists('resized'): #경로가 없으면
    os.mkdir('resized') # resized 폴더 생성
    for label, img_paths in dataset.items(): # resized > 가수명 폴더 생성
        os.mkdir(f'resized/{label}')
else:
    pass

for label, img_paths in dataset.items():
    for img_path in img_paths:
        img_re = resize_img(img_path)
        cv2.imwrite(f'resized/{img_path[64:]}', img_re) # resized/{가수명}/{파일명} 저장

## 2. YOLO-WORLD Bounding Box 추출
- [YOLO-WORLD](https://github.com/AILab-CVC/YOLO-World) 활용해 스타 사진의 Face Detection
- [스타 리스트](https://docs.google.com/spreadsheets/d/13ni1cFJ_IeJZuY2Fb2H6pioZMOPeCSEjV40H0a-bEos/edit?usp=drive_link)



### YOLO-WORLD 불러오기

In [ ]:
!git clone --recursive https://github.com/AILab-CVC/YOLO-World
%cd YOLO-World/

Cloning into 'YOLO-World'...
remote: Enumerating objects: 300, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 300 (delta 74), reused 50 (delta 50), pack-reused 203
Receiving objects: 100% (300/300), 538.38 KiB | 9.28 MiB/s, done.
Resolving deltas: 100% (139/139), done.
Submodule 'third_party/mmyolo' (https://github.com/onuralpszr/mmyolo.git) registered for path 'third_party/mmyolo'
Cloning into '/content/YOLO-World/third_party/mmyolo'...
remote: Enumerating objects: 4944, done.        
remote: Counting objects: 100% (1412/1412), done.        
remote: Compressing objects: 100% (288/288), done.        
remote: Total 4944 (delta 1203), reused 1128 (delta 1124), pack-reused 3532        
Receiving objects: 100% (4944/4944), 3.59 MiB | 15.86 MiB/s, done.
Resolving deltas: 100% (3195/3195), done.
Submodule path 'third_party/mmyolo': checked out '4d97b3a06609dba94b8ec584be2f2029cfdb7519'
/content/YOLO-World


In [ ]:
import os
# Install certain version of requests,tqdm,rich for openxlab (fix for yolo_world)
# Install mmcv before avoding compiling of mmcv and shortining waiting time installs "whl" file
if 'COLAB_GPU' in os.environ:
  !pip install requests==2.28.2 tqdm==4.65.0 rich==13.4.2
  %pip install -U openmim
  !mim install "mmengine>=0.7.0"
  !mim install "mmcv"
else:
  !pip install torch wheel requests==2.28.2 tqdm==4.65.0 rich==13.4.2


!pip install -e . -vv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 14.1 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.2
    Uninstalling tqdm-4.66.2:
      Successfully uninstalled tqdm-4.66.2
  Attempting uninstall: rich
    Found existing installation: rich 13.7.0
    Uninstalling rich-13.7.0:
      Successfully uninstalled rich-13.7.0
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 843.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 21.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.1/953.1 kB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.0/443.0 kB 39.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for oss2: filename=oss2-2.17.0-py3-none-any.whl size=112371 sha256=d11e377d54541dc81ab82acd20dea09251a7d517760f3fbb4f1bbfb4cb60e093
  Stored in directory: /root/.cache/pip/wheels/87/04/7b/7e61b8157fdf211c513

Looking in links: https://download.openmmlab.com/mmcv/dist/cu121/torch2.1.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.7/451.7 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 8.9 MB/s eta 0:00:00
Looking in links: https://download.openmmlab.com/mmcv/dist/cu121/torch2.1.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.1/94.1 MB 3.1 MB/s eta 0:00:00
Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
Non-user install because site-packages writeable
Created temporary directory: /tmp/pip-build-tracker-_zqepz_m
Initialized build tracking at /tmp/pip-build-tracker-_zqepz_m
Created build tracker: /tmp/pip-build-tracker-_zqepz_m
Entered build tracker: /tmp/pip-build-tracker-_zqepz_m
Created temporary directory: /tmp/pip-install-c73d_chw
Created temporary directory: /tmp/pip-ephem-wheel-cache-flsq4c01
Obtaining file:///content/YOLO-World
  Added file:///content/YOLO-World to build tracker '/tmp/pip-b

In [ ]:
# Restart collab session (required for mmengine)
quit()

In [ ]:
!wget https://huggingface.co/spaces/stevengrove/YOLO-World/resolve/main/yolow-v8_l_clipv2_frozen_t2iv2_bn_o365_goldg_pretrain.pth?download=true
!mv yolow-v8_l_clipv2_frozen_t2iv2_bn_o365_goldg_pretrain.pth?download=true yolow-v8_l_clipv2_frozen_t2iv2_bn_o365_goldg_pretrain.pth
!wget https://huggingface.co/spaces/stevengrove/YOLO-World/resolve/main/configs/pretrain/yolo_world_l_t2i_bn_2e-4_100e_4x8gpus_obj365v1_goldg_train_lvis_minival.py?download=true
!mv yolo_world_l_t2i_bn_2e-4_100e_4x8gpus_obj365v1_goldg_train_lvis_minival.py?download=true yolo_world_l_t2i_bn_2e-4_100e_4x8gpus_obj365v1_goldg_train_lvis_minival.py
!wget https://media.roboflow.com/notebooks/examples/dog.jpeg
!cp -r yolo_world_l_t2i_bn_2e-4_100e_4x8gpus_obj365v1_goldg_train_lvis_minival.py /content/YOLO-World/configs/pretrain/

--2024-02-28 09:52:19--  https://huggingface.co/spaces/stevengrove/YOLO-World/resolve/main/yolow-v8_l_clipv2_frozen_t2iv2_bn_o365_goldg_pretrain.pth?download=true
Resolving huggingface.co (huggingface.co)... 65.8.178.27, 65.8.178.12, 65.8.178.118, ...
Connecting to huggingface.co (huggingface.co)|65.8.178.27|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/cc/e2/cce2cecbe61f2002bfd047dbb19558446e78db43c2b1c4a89071860f6eeb9a0e/c86e28e6f248ba6d5f5f83137f6968d53982c428f55e2a8c6d4edfa810c60658?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27yolow-v8_l_clipv2_frozen_t2iv2_bn_o365_goldg_pretrain.pth%3B+filename%3D%22yolow-v8_l_clipv2_frozen_t2iv2_bn_o365_goldg_pretrain.pth%22%3B&Expires=1709373140&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwOTM3MzE0MH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2NjL2UyL2NjZTJjZWNiZTYxZjIwMDJiZm

In [ ]:
import torch
from mmengine.config import Config
from mmengine.dataset import Compose
from mmengine.runner import Runner
from mmengine.runner.amp import autocast
from mmyolo.registry import RUNNERS
from torchvision.ops import nms

In [ ]:
if __name__ == "__main__":
    # load config
    cfg = Config.fromfile(
        "/content/YOLO-World/configs/pretrain/yolo_world_l_t2i_bn_2e-4_100e_4x8gpus_obj365v1_goldg_train_lvis_minival.py"
    )
    cfg.work_dir = "."
    cfg.load_from = "yolow-v8_l_clipv2_frozen_t2iv2_bn_o365_goldg_pretrain.pth"
    runner = Runner.from_cfg(cfg)
    runner.call_hook("before_run")
    runner.load_or_resume()
    pipeline = cfg.test_dataloader.dataset.pipeline
    runner.pipeline = Compose(pipeline)
    runner.model.eval()

02/28 09:52:49 - mmengine - WARNING - Failed to search registry with scope "mmyolo" in the "log_processor" registry tree. As a workaround, the current "log_processor" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmyolo" is a correct scope, or whether the registry is initialized.
02/28 09:52:49 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
    CUDA available: False
    MUSA available: False
    numpy_random_seed: 771598735
    GCC: x86_64-linux-gnu-gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
    PyTorch: 2.1.0+cu121
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.1.1 (Git Hash 

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


02/28 09:53:07 - mmengine - INFO - Distributed training is not used, all SyncBatchNorm (SyncBN) layers in the model will be automatically reverted to BatchNormXd layers if they are used.
02/28 09:53:07 - mmengine - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) RuntimeInfoHook                    
(49          ) EMAHook                            
(BELOW_NORMAL) LoggerHook                         
 -------------------- 
after_load_checkpoint:
(49          ) EMAHook                            
 -------------------- 
before_train:
(9           ) YOLOv5ParamSchedulerHook           
(VERY_HIGH   ) RuntimeInfoHook                    
(49          ) EMAHook                            
(NORMAL      ) IterTimerHook                      
(VERY_LOW    ) CheckpointHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(NORMAL      ) DistSamplerSeedHo

### B-Box 검출 함수
- class_names: 인식할 클래스 지정
- return svimage[:, :, ::-1]: 이미지로 출력
- return detections: bbox, confidensce 로 출력

In [ ]:
import PIL.Image
import supervision as sv
import numpy as np

bounding_box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator(text_position=sv.Position.CENTER)

class_names = ("face")

# class_names2 = ("person")

def run_image(
        runner,
        input_image,
        max_num_boxes=100,
        score_thr=0.05,
        nms_thr=0.5,
        output_image="output.png",
):
    global xyxy
    texts = [[t.strip()] for t in class_names.split(",")] + [[" "]]
    data_info = runner.pipeline(dict(img_id=0, img_path=input_image,
                                     texts=texts))

    data_batch = dict(
        inputs=data_info["inputs"].unsqueeze(0),
        data_samples=[data_info["data_samples"]],
    )

    with autocast(enabled=False), torch.no_grad():
        output = runner.model.test_step(data_batch)[0]
        runner.model.class_names = texts
        pred_instances = output.pred_instances

    keep_idxs = nms(pred_instances.bboxes, pred_instances.scores, iou_threshold=nms_thr)
    pred_instances = pred_instances[keep_idxs]
    pred_instances = pred_instances[pred_instances.scores.float() > score_thr]

    if len(pred_instances.scores) > max_num_boxes:
        indices = pred_instances.scores.float().topk(max_num_boxes)[1]
        pred_instances = pred_instances[indices]
    output.pred_instances = pred_instances

    pred_instances = pred_instances.cpu().numpy()
    detections = sv.Detections(
        xyxy=pred_instances['bboxes'],
        class_id=pred_instances['labels'],
        confidence=pred_instances['scores']
    )

    labels = [
        f"{class_id} {confidence:0.2f}"
        for class_id, confidence
        in zip(detections.class_id, detections.confidence)
    ]

    image = PIL.Image.open(input_image)
    svimage = np.array(image)
    svimage = bounding_box_annotator.annotate(svimage, detections)
    svimage = label_annotator.annotate(svimage, detections, labels)
    # return svimage[:, :, ::-1]
    return detections

### xyxy만 출력
- 위 디텍팅 함수에서 출력된 다양한 정보가 포함된 문자열을 [[xyxy], [xyxy]] 만 남게 전처리하는 코드
- xyxy가 없을 시 또는 깨진 문자열로 출력되는 경우는 예외처리함

In [ ]:
import numpy as np

def extract_xyxy(detections_str):

    # 'xyxy'의 시작과 끝 위치 찾기
    start_index = detections_str.find("xyxy=array([[") + len("xyxy=array([[")
    end_index = detections_str.find("]]")

    # 숫자 부분 추출
    xyxy_str = detections_str[start_index:end_index]

    # 문자열을 리스트로 분할합니다.
    boxes_str = xyxy_str.split("],\n       [")

    # 각 바운딩 박스 문자열을 좌표 리스트로 변환합니다.
    xyxy_list = []
    for box_str in boxes_str:
        coords = [float(coord.strip()) for coord in box_str.split(",")]
        xyxy_list.append(coords)

    return xyxy_list

### 스타 이름 인덱스화

In [ ]:
import csv

star_names = {}

with open("/content/drive/MyDrive/객체인식_3조/스타 얼굴 인식 모델/스타 이미지 데이터/스타 수집 대상 - 시트1.csv", mode="r", encoding="utf-8") as inp:
    reader = csv.reader(inp)
    star_names = {rows[3]: rows[0] for rows in reader}
    del star_names['검색어']

print(star_names)

{'라이즈 원빈': '0', '차은우': '1', '라이즈 앤톤': '2', '라이즈 성찬': '3', '슈퍼주니어 규현': '4', '라이즈 소희': '5', '하이라이트 이기광': '6', '라이즈 은석': '7', '워너원 박지훈': '8', '방탄소년단 지민': '9', '라이즈 쇼타로': '10', '강다니엘': '11', '엑소 백현': '12', '세븐틴 원우': '13', '세븐틴 민규': '14', '세븐틴 정한': '15', '더보이즈 에릭': '16', '방탄소년단 정국': '17', '방탄소년단 뷔': '18', '슈퍼주니어 희철': '19', '워너원 황민현': '20', '동방신기 최강창민': '21', '세븐틴 조슈아': '22', '세븐틴 에스쿱스': '23', '방탄소년단 RM': '24', '워너원 김재환': '25', '더보이즈 주연': '26', '동방신기 유노윤호': '27', '샤이니 키': '28', '세븐틴 도겸': '29', '세븐틴 우지': '30', '방탄소년단 진': '31', '세븐틴 호시': '32', '세븐틴 승관': '33', '세븐틴 준': '34', '세븐틴 디노': '35', 'NCT 도영': '36', 'NCT 재현': '37', '세븐틴 디에잇': '38', '방탄소년단 제이홉': '39', '방탄소년단 슈가': '40', '세븐틴 버논': '41', 'NCT 마크': '42', '더보이즈 선우': '43', '옹성우': '44', '송민호': '45', '더보이즈 현재': '46', 'NCT 해찬': '47', '하이라이트 윤두준': '48', 'NCT 정우': '49', '위너 김진우': '50', '제로베이스원 성한빈': '51', '슈퍼주니어 시원': '52', '슈퍼주니어 동해': '53', '샤이니 민호': '54', 'NCT 태용': '55', '투모로우바이투게더 연준': '56', '더보이즈 영훈': '57', '비투비 이창섭': '58', '투모로우바이투게더 수빈': '59', 

### 전체 이미지 B-Box 검출 후 .txt 파일 저장
- xyxy -> 상대좌표화된 xywh 으로 변경

In [ ]:
import re
import os
import numpy as np
from unicodedata import normalize

folder_path = '/content/drive/MyDrive/객체인식_3조/스타 얼굴 인식 모델/스타 이미지 데이터/resized'

for label in os.listdir(folder_path):
    sub_path = folder_path+'/'+label+'/' # /content/resized/naver_트와이스 지효/
    label_name = normalize('NFC', label[6:]) # naver_트와이스 지효 -> 트와이스 지효
    star_index = int(star_names[label_name]) # 트와이스 지효 -> 0

    for filename in os.listdir(sub_path):
        if 'jpg' in filename:
          img = run_image(runner, sub_path+filename) # /content/resized/naver_트와이스 지효/167.jpg
          input_string = str(img)
          bbox_txt = ''

          try:
            number_list = extract_xyxy(input_string) # [x,y,x,y]
            for i in range(len(number_list)):
              # Calculate x center, y center
              x_center = ((number_list[i][0] + number_list[i][2]) / 2) / 640
              y_center = ((number_list[i][1] + number_list[i][3]) / 2) / 640

              # Calculate width and height
              width = (number_list[i][2] - number_list[i][0]) / 640
              height = (number_list[i][3] - number_list[i][1]) / 640
              bbox_txt += f"{star_index} {x_center} {y_center} {width} {height}\n"

            txt_path = sub_path + filename[:-4] + '.txt' # 파일명.txt 로 같은 폴더에 저장
            with open(txt_path, "w") as file:
                file.write(bbox_txt)
                file.close()
          except ValueError:
            continue

print("Bounding boxes saved to bounding_boxes.txt")

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Bounding boxes saved to bounding_boxes.txt


In [ ]:
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

# random_star = random.choice(resized_star)
# idx = random.randint(0, 500)

# **Assumptions**
image_file = f"/content/drive/MyDrive/객체인식_3조/스타 얼굴 인식 모델/스타 이미지 데이터/resized/naver_블랙핑크 제니/245_1.jpg"   # Replace with your image filename
label_file = f"/content/drive/MyDrive/객체인식_3조/스타 얼굴 인식 모델/스타 이미지 데이터/resized/naver_블랙핑크 제니/245_1.txt"

# **1. Read the label data**
with open(label_file, "r") as f:
    for line in f:
        data = line.strip().split()
        label = data[0]
        x_center = float(data[1])
        y_center = float(data[2])
        width = float(data[3])
        height = float(data[4])

# **2. Load the image**
image = cv2.imread(image_file)

# **3. Calculate bounding box coordinates**
x_min = int(x_center - (width / 2)) * 640
y_min = int(y_center - (height / 2)) * 640
x_max = int(x_center + (width / 2)) * 640
y_max = int(y_center + (height / 2)) * 640

# **4. Draw the bounding box and label**
cv2.rectangle(image, (x_min, y_min), (x_max, y_max), color=(0, 255, 0), thickness=2)  # Green color

# Select a font (This might require installing a font or using a default path)
# font_path = '/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf'
# font = ImageFont.truetype(font_path, size=16)

# Use PIL to add text (OpenCV's text rendering can be less flexible)
image_pil = Image.fromarray(image)
draw = ImageDraw.Draw(image_pil)
draw.text((x_min, y_min - 20), label, fill=(0, 255, 0, 255))  # Green color
image = cv2.cvtColor(np.array(image_pil), cv2.COLOR_RGB2BGR)

# **5. Display the image in Jupyter Notebook**
plt.figure(figsize=(10, 8))  # Adjust figure size as needed
plt.imshow(image)
plt.show()